In [1]:
import tensorflow as tf
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

import numpy as np
import pickle

data_files = ['dev', 'test', 'train']

data_path = "./data/%s_dataset.json"
x_file = './data/agg_data/%s_agg_X.pkl'
y_file = './data/agg_data/%s_agg_Y.pkl'
idx_file = './data/op_data/%s_op_idx.pkl'

def get_data(dt):
    with open(y_file%dt, 'rb') as fy:
        _Y = pickle.load(fy)

    with open(x_file%dt, 'rb') as fx:
        _X = pickle.load(fx)
    return (_X, _Y)

dev_X, dev_Y = get_data('dev')


def get_batch(idx, batch_size):
    lw = idx * batch_size
    hs = lw + batch_size
    if hs < len(dev_X):
        return (dev_X[lw:hs][:], dev_Y[lw:hs][:])
    else:
        return (dev_X[lw:][:], dev_Y[lw:][:])

In [1]:
# def train():
    n_input = 1500
    n_hidden1 = 1024
    n_hidden2 = 512
    n_hidden3 = 64
    n_output = 6

    learning_rate = 1e-4
    n_iteration = 11
    batch_size = 128
    dropout = 0.1

    X = tf.placeholder("float", [None, n_input])
    Y = tf.placeholder("float", [None, n_output])
    # keep_prob = tf.placeholder(tf.float32) # used to control the dropout rate

    weights = {
    'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
    'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
    'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1))
    }

    biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
    'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
    'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
    }

    layer_1 = tf.nn.tanh(tf.add(tf.matmul(X, weights['w1']), biases['b1']))
    layer_2 = tf.nn.tanh(tf.add(tf.matmul(layer_1, weights['w2']), biases['b2']))
    layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, weights['w3']), biases['b3']))
    # layer_drop = tf.nn.dropout(layer_3, keep_prob)
    output_layer = tf.nn.leaky_relu(tf.add(tf.matmul(layer_3, weights['out']), biases['out']))

    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output_layer))
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

    correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)



    n_batches = len(dev_X)//batch_size

    # train on mini batches
    for i in range(n_iteration):
        for j in range(n_batches+1):
            (batch_x, batch_y) = get_batch(j, batch_size)
            sess.run(train_step, feed_dict={X: batch_x, Y: batch_y})

            # print loss and accuracy (per minibatch)
        if i % 5 == 0:
            minibatch_loss, minibatch_accuracy = sess.run([cross_entropy, accuracy], feed_dict={X: batch_x, Y: batch_y})
            print("Iteration", str(i), "\t| Batch", str(j), "\t| Loss =", str(minibatch_loss), "\t| Accuracy =", str(minibatch_accuracy))

    tf.train.Saver().save(sess, './checkpoints/agg/training.ckpt')

    test_X, test_Y = get_data('test')
    test_accuracy = sess.run(accuracy, feed_dict={X: test_X, Y: test_Y})
    print("\nAccuracy on test set : ", test_accuracy)

In [2]:
train()

Iteration 0 	| Batch 19 	| Loss = 1.4073397 	| Accuracy = 0.5890411
Iteration 5 	| Batch 19 	| Loss = 0.411447 	| Accuracy = 0.8767123
Iteration 10 	| Batch 19 	| Loss = 0.2886355 	| Accuracy = 0.91780823

Accuracy on test set :  0.7566709


In [2]:
def test():
    n_input = 1500
    n_hidden1 = 1024
    n_hidden2 = 512
    n_hidden3 = 64
    n_output = 6

    learning_rate = 1e-4
    n_iteration = 11
    batch_size = 128
    dropout = 0.1

    X = tf.placeholder("float", [None, n_input])
    Y = tf.placeholder("float", [None, n_output])
    # keep_prob = tf.placeholder(tf.float32) # used to control the dropout rate

    weights = {
    'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
    'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
    'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1))
    }

    biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
    'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
    'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
    }

    layer_1 = tf.nn.tanh(tf.add(tf.matmul(X, weights['w1']), biases['b1']))
    layer_2 = tf.nn.tanh(tf.add(tf.matmul(layer_1, weights['w2']), biases['b2']))
    layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, weights['w3']), biases['b3']))
    # layer_drop = tf.nn.dropout(layer_3, keep_prob)
    output_layer = tf.nn.leaky_relu(tf.add(tf.matmul(layer_3, weights['out']), biases['out']))

    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output_layer))
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

    correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)

    tf.train.Saver().restore(sess, './checkpoints/agg/training.ckpt')
    test_X, test_Y = get_data('test')

    test_accuracy = sess.run(accuracy, feed_dict={X: test_X, Y: test_Y})
    print("\nAccuracy on test set : ", test_accuracy)

In [4]:
test()

INFO:tensorflow:Restoring parameters from ./checkpoints/agg/training.ckpt


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key Variable_10 not found in checkpoint
	 [[node save_1/RestoreV2 (defined at <ipython-input-2-09c095889e98>:47)  = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_1/Const_0_0, save_1/RestoreV2/tensor_names, save_1/RestoreV2/shape_and_slices)]]

Caused by op 'save_1/RestoreV2', defined at:
  File "/root/installation/miniconda3/envs/tf_nlp/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/root/installation/miniconda3/envs/tf_nlp/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/root/installation/miniconda3/envs/tf_nlp/lib/pytho


Accuracy on test set :  0.7583651


In [30]:
import tensorflow as tf
import numpy as np
import pickle
class AggNN():
    def __init__(self):
        n_input = 1500
        n_hidden1 = 1024
        n_hidden2 = 512
        n_hidden3 = 64
        n_output = 6
        
        self.learning_rate = 1e-4
        self.n_iteration = 10
        self.batch_size = 128
        
        self.X = tf.placeholder("float", [None, n_input])
        self.Y = tf.placeholder("float", [None, n_output])
        
        self.weights = {
            'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
            'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
            'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
            'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1))
        }
        
        self.biases = {
            'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
            'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
            'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
            'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
        }
        
        self.layer_1 = tf.nn.tanh(tf.add(tf.matmul(self.X, self.weights['w1']), self.biases['b1']))
        self.layer_2 = tf.nn.tanh(tf.add(tf.matmul(self.layer_1, self.weights['w2']), self.biases['b2']))
        self.layer_3 = tf.nn.relu(tf.add(tf.matmul(self.layer_2, self.weights['w3']), self.biases['b3']))
        # layer_drop = tf.nn.dropout(layer_3, keep_prob)
        self.output_layer = tf.nn.leaky_relu(tf.add(tf.matmul(self.layer_3, self.weights['out']), self.biases['out']))
        
        self.cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.Y, logits=self.output_layer))
        self.train_step = tf.train.AdamOptimizer(self.learning_rate).minimize(self.cross_entropy)
        
        self.correct_pred = tf.equal(tf.argmax(self.output_layer, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_pred, tf.float32))
        
        self.dev_X, self.dev_Y = self.get_data('train')
        self.n_batches = len(self.dev_X)//self.batch_size
        ############################################
        self.init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(self.init)
        tf.train.Saver().restore(self.sess, './checkpoints/agg/training.ckpt')
#         for i in range(self.n_iteration):
#             for j in range(self.n_batches+1):
#                 (batch_x, batch_y) = self.get_batch(j, self.batch_size)
#                 self.sess.run(self.train_step, feed_dict={self.X: batch_x, self.Y: batch_y})

#                 # print loss and accuracy (per minibatch)
#             if i % 5 == 0:
#                 minibatch_loss, minibatch_accuracy = \
#                     self.sess.run([self.cross_entropy, self.accuracy], feed_dict={self.X: batch_x, self.Y: batch_y})
#                 print("Iteration", str(i), "\t| Loss =", str(minibatch_loss), "\t| Accuracy =", str(minibatch_accuracy))
#         self.store_param()
        self.test()
        
    def get_data(self, dt):
        x_file = './data/agg_data/%s_agg_X.pkl'
        y_file = './data/agg_data/%s_agg_Y.pkl'
        with open(y_file%dt, 'rb') as fy:
            _Y = pickle.load(fy)

        with open(x_file%dt, 'rb') as fx:
            _X = pickle.load(fx)
        return (_X, _Y)
    
    def get_batch(self, idx, batch_size):
        lw = idx * batch_size
        hs = lw + batch_size
        if hs < len(self.dev_X):
            return (self.dev_X[lw:hs][:], self.dev_Y[lw:hs][:])
        else:
            return (self.dev_X[lw:][:], self.dev_Y[lw:][:])
    
    def load_param(self):
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)
        tf.train.Saver().restore(sess, './checkpoints/agg/training.ckpt')
        return sess
        
    def store_param(self):
        tf.train.Saver().save(self.sess, './checkpoints/agg/training.ckpt')
        
    def test(self):
        test_X, test_Y = self.get_data('test')
        test_accuracy = self.sess.run(self.accuracy, feed_dict={self.X: test_X, self.Y: test_Y})
        print("\nAccuracy on test set : ", test_accuracy)
        
    def train(self):
        # train on mini batches
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)
        for i in range(self.n_iteration):
            for j in range(self.n_batches+1):
                (batch_x, batch_y) = self.get_batch(j, self.batch_size)
                sess.run(self.train_step, feed_dict={self.X: batch_x, self.Y: batch_y})

                # print loss and accuracy (per minibatch)
            if i % 5 == 0:
                minibatch_loss, minibatch_accuracy = \
                    sess.run([self.cross_entropy, self.accuracy], feed_dict={self.X: batch_x, self.Y: batch_y})
                print("Iteration", str(i), "\t| Loss =", str(minibatch_loss), "\t| Accuracy =", str(minibatch_accuracy))
        self.store_param(sess)
        self.test(sess)

In [31]:
agg_nn = AggNN()

INFO:tensorflow:Restoring parameters from ./checkpoints/agg/training.ckpt


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key Variable_80 not found in checkpoint
	 [[node save_9/RestoreV2 (defined at <ipython-input-30-364d13744aca>:51)  = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_9/Const_0_0, save_9/RestoreV2/tensor_names, save_9/RestoreV2/shape_and_slices)]]

Caused by op 'save_9/RestoreV2', defined at:
  File "/root/installation/miniconda3/envs/tf_nlp/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/root/installation/miniconda3/envs/tf_nlp/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/root/installation/miniconda3/envs/tf_nlp/lib/pyth

In [22]:
agg_nn.train()

Iteration 0 	| Loss = 0.7279095 	| Accuracy = 0.7777778
Iteration 5 	| Loss = 0.5288819 	| Accuracy = 0.85858583

Accuracy on test set :  0.76725966


In [25]:
sess = agg_nn.load_param()

INFO:tensorflow:Restoring parameters from ./checkpoints/agg/training.ckpt


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key Variable_56 not found in checkpoint
	 [[node save_7/RestoreV2 (defined at <ipython-input-23-ff0b4c3d6d39>:70)  = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_7/Const_0_0, save_7/RestoreV2/tensor_names, save_7/RestoreV2/shape_and_slices)]]

Caused by op 'save_7/RestoreV2', defined at:
  File "/root/installation/miniconda3/envs/tf_nlp/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/root/installation/miniconda3/envs/tf_nlp/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/root/installation/miniconda3/envs/tf_nlp/lib/pyth